# ***1. Load Data and setting for Korean***

In [ ]:
## matplotlib 사용시 한국어 나오게 하는 코드

!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)

fm._rebuild()

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic') 


In [ ]:
import pandas as pd #판다스 패키지 불러오기
import lightgbm as lgb
import folium
import numpy as np

train = pd.read_csv("/content/drive/My Drive/Dacon/Data/Prediction of arriving bus time/train.csv")
test = pd.read_csv("/content/drive/My Drive/Dacon/Data/Prediction of arriving bus time/test.csv")
submission=pd.read_csv("/content/drive/My Drive/Dacon/Data/Prediction of arriving bus time/submission_제출양식.csv")

# ***2. Exploratory data analysis(EDA)***

In [ ]:
train.head()

+ id : 고유 id
+ date : 버스 운행 날짜
+ route_id : 버스 노선 ID
+ vh_id : 버스 id
+ route_nm : 버스 노선 실제 번호
+ now_latitude : 현재 정류소의 위도
+ now_longitude : 현재 정류소의 경도
+ now_station : 현재 정류소 이름
+ now_arrive_time : 현재 정류장에 도착한 시간
+ distance : 현재 정류장에서 다음 정류장까지 실제 이동한 거리
+ next_station : 다음 정류소 이름
+ next_latitude : 다음 정류소의 위도
+ next_longitude : 다음 정류소의 경도
+ next_arrive_time : 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)으로 답안 제출을 위해서 예측해야 되는 값

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
print(train['date'].min())
print(train['date'].max())

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.isnull().sum() #결측치 확인법 1

In [ ]:
train.info() #결측치 확인법 2

In [ ]:
train.describe()

In [ ]:
train.head()

In [ ]:
train[['route_id', 'vh_id']].drop_duplicates().groupby('route_id').count().head() #버스 노선과 버스번호가 중복된것들 드롭& 같은 버스 노선별로 버스 배차량 확인

In [ ]:
train[['route_id','next_arrive_time']].groupby('route_id').mean() #노선별로 걸리는 시간이 상이함을 알수있음

folium.map()

*  location 옵션에 있는 좌표를 중심으로 지도 시각화



In [ ]:
coord = [35.1559361, 129.1384361] #지도의 중심부가 될 좌표(전세계 맵을 다 지원함)
map_osm = folium.Map(location = coord, zoom_start=12) #default 값은 10(숫자가 클수록 확대됨)
folium.Marker(coord, popup='busan university', tooltip = 'busan university tooltip').add_to(map_osm) # 마커 생성 뒤 지도에 추가해주는 기능
map_osm.save('index.html')

map_osm

In [ ]:
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
map_bus_route = map_data[map_data['route_id'] == 405136521].drop_duplicates("now_station") # 각 정류장간격의 시간이 가장 오래걸렷던 경로

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 12)

for item in map_bus_route.index:
    lat = map_bus_route.loc[item,'now_latitude'] #해당 경로에 대한 위도
    long = map_bus_route.loc[item,'now_longitude'] #해당 경로에 대한 경도
    
    folium.Marker([lat,long], popup = map_bus_route.loc[item,'now_station'],
                      icon = folium.Icon(color = 'red', icon = 'info-sign')
                      ).add_to(map_osm)

map_osm.save('index.html')

map_osm

예상대로 오래걸리는 루트는 번잡한 제주시 한복판

# ***3.Data Cleansing & Pre-Processing***

pd.Series.str.slice()

> Series 내에 문자열을 일괄적으로 slicing (자르기)


In [ ]:
train['now_arrive_time']

In [ ]:
train['now_arrive_time']=train['now_arrive_time'].str.slice(stop=-1).astype(int) #마지막 문자 전까지 자르기 ('시'전까지 슬라이싱) & int 형으로 바꿔주기
test['now_arrive_time']=test['now_arrive_time'].str.slice(stop=-1).astype(int)
train.head()

In [ ]:
test.info() # now_arrive_time 이 object에서 int 로 바뀌어있음을 확인

# ***4. Feature Engineering & Initial Modeling***

In [ ]:
features= ['now_latitude','now_longitude','now_arrive_time','distance']
target= ['next_arrive_time']

X_train, X_test, y_train= train[features], test[features], train[target]
print(X_test)

# ***5. Model Tuning & Evaluation***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

model_dict={ 'linear': LinearRegression(),
            'rf': RandomForestRegressor(random_state=0, n_jobs=-1),
            'lgbm': lgb.LGBMRegressor(random_state=0)}
print(model_dict['linear'])

print(X_train.shape)
print(y_train.shape)

In [ ]:
model_result={}

for key in model_dict.keys():

    model_dict[key].fit(X_train,y_train)

    model_result[key] = model_dict[key].predict(X_test)

model_result

In [ ]:
lr_submit= submission.copy()
rf_submit= submission.copy()
lgbm_submit= submission.copy()

In [ ]:
lgb

In [ ]:
lr_submit['next_arrive_time'] = model_result['linear']
rf_submit['next_arrive_time'] = model_result['rf']
lgbm_submit['next_arrive_time'] = model_result['lgbm']

In [ ]:
lr_submit.to_csv('lr_submit.csv', index= False)
rf_submit.to_csv('rf_submit.csv', index= False)
lgbm_submit.to_csv('lgbm_submit.csv', index= False)